In [19]:
import numpy as np
import pickle

In [20]:
path_pyspi = "main_files/crab_fits/only_weak_pulsar/0043_4_5/pl_w_p_50_100/source_parameters.pickle"
path_smf = "main_files/spimodfit_fits/0043_4_5/crab_med_energy_pl_fit.pickle"

In [29]:
with open(f"{path_pyspi}", "rb") as f:
    pyspi_val, pyspi_cov = pickle.load(f)
    
with open(f"{path_smf}", "rb") as f:
    smf_val, smf_cov = pickle.load(f)

pyspi_val, smf_val

(array([ 7.04188104e-04, -2.07821939e+00,  2.35104652e-08, -3.49777828e+00]),
 array([ 6.73557456e-04, -2.11996004e+00]))

In [30]:
np.sqrt(pyspi_cov), np.sqrt(smf_cov)

/tmp/ipykernel_25263/3762676499.py:1: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(pyspi_cov), np.sqrt(smf_cov)


(array([[3.99257198e-06, 2.22380289e-04, 1.10166148e-07,            nan],
        [2.22380289e-04, 1.45186929e-02,            nan,            nan],
        [1.10166148e-07,            nan, 5.14966562e-07, 4.72757070e-04],
        [           nan,            nan, 4.72757070e-04, 2.26295548e+00]]),
 array([[3.34797161e-06, 1.97489284e-04],
        [1.97489284e-04, 1.38441392e-02]]))

In [26]:
def powerlaw_binned_spectrum(energy_bins, spectrum):
    assert np.amin(energy_bins) > 0, "All energy bin values must be greater 0"
    assert np.amin(spectrum) > 0, "All spectrum values must be greater 0"
    
    B = np.log(spectrum[1:] / spectrum[:-1]) / np.log(energy_bins[1:] / energy_bins[:-1])
    A = spectrum[:-1] / (energy_bins[:-1] ** B)
    
    C = B + 1.
    # prevent rounding errors
    C[np.abs(C) < 1e-7] = 0
    
    regular = np.nonzero(C)[0]
    non_regular = []
    for i in range(len(energy_bins) - 1):
        if not i in regular:
            non_regular.append(i)
    # non_regular = np.array(non_regular)
    
    binned_spectrum = np.zeros(len(energy_bins) - 1)
    binned_spectrum[regular] = A[regular] / (C[regular]) * (energy_bins[regular+1]**(C[regular]) - energy_bins[regular]**(C[regular]))
    # binned_spectrum[non_regular] = A[non_regular] * (np.log(energy_bins[non_regular+1]) - np.log(energy_bins[non_regular]))
    
    return binned_spectrum

In [23]:
def pl(norm, index, x):
    return norm*(x/100)**index

In [24]:
pyspi_50 = pl(pyspi_val[0], pyspi_val[1], 50)
smf_50 = pl(smf_val[0], smf_val[1], 50)
pyspi_50, smf_50

(0.0029736856585033875, 0.0029278323813110583)

In [27]:
powerlaw_binned_spectrum(
    np.array([50, 100]),
    np.array([pyspi_50, pyspi_val[0]])
)

array([0.07258771])

In [28]:
powerlaw_binned_spectrum(
    np.array([50, 100]),
    np.array([smf_50, smf_val[0]])
)

array([0.07057026])